# Adaptive Sampling

In this tutorial you will learn:
    
1. Basic acquisition using RunEngine
2. Generating basic acquistion plans for multiple sample enviornments on a simulated diffraction beamline
3. How to use the Bluesky Adaptive harness to readly integrate AI-agents with the beamline
4. Demonstration of Reinforcement Learning (RL) being used to optimize data collection stradegies


For more bluesky tutorials, goto https://try.nsls2.bnl.gov/

<img src="BS_layout.png" alt="Bluesky flow diagram" style="width: 600px;"/>  [(image source)](https://iopscience.iop.org/article/10.1088/2632-2153/abc9fc)

## Some of the software we use:

* **Bluesky RunEngine** for experiment orchestration (sequencing)
* **Bluesky Ophyd** for device integration (for this demo, a simulated detector)
* **Bluesky Widgets** components for live-updating ("streaming") visualization
* **Matplotlib** for visualization
* **Bluesky Adaptive**, an adaptive "harness" for integrating an Agent in a feedback loop with the Bluesky RunEngine
* **Tensorflow** for the model

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt


In [ ]:
from bluesky import RunEngine
from bluesky.plans import count
from utils.simulated_hardware import detector, sample_selector, select_sample
from utils.visualization import stream_to_figures
from utils.adaptive_recommendations import with_agent

detector.delay = .1

## Make a "RunEngine"

* Processes a set of instructions (a "plan") from the user
* Direct hardware, tracks what is moving when, and tries to clean up correct in the event of success or failures
* Emits metadata and data in a streaming fashion for consumers (plots, models, storage, etc.)

In [ ]:
RE = RunEngine()

The RunEngine can move this things, and it can take data.

## Acquire some images

For this tutorial, we will be moving a <code>sample_selector</code> ophyd device, which can switch between samples on our simulated beamline.  We can read the current status of the motor to see which sample we are currently on.

In [ ]:
sample_selector.read()

To move this motor, we could use the built-in bluesky <code>mv</code> plan...

In [ ]:
from bluesky.plan_stubs import mv

RE(mv(sample_selector,2))

print (sample_selector.read())

Or, we can write our own custom plan with whatever language and extensions we wish.

In [ ]:
def select_sample(sample_number):
    print ('moving to sample '+str(sample_number))
    yield from mv(sample_selector, sample_number)

In [ ]:
RE(select_sample(0))  # This moves a sample positioner to place Sample 0 in the beam.

In this tutorial, we are focused on streaming data formatted in the document-model.  As such, we're going to setup our visualization first, and then stream data in as we are measuring.  

Functionally, this involve creating a figure and axes with Matplotlib, and then passing these to a callback function that will digest documents emitted by the RunEngine.

When the cell below is first run, you should see a checkerboard pattern and some basic metadata in the title (sample number, and measurment number).

In [ ]:
fig, axes = plt.subplots(squeeze=False, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

Now we can select the sample we want and pass a <code>count</code> plan into the run engine.   

<code>count</code> is the most basic acquisition plan in Bluesky.  It takes as an argument a list of detectors.  In this case, we will pass it both the <code>detector</code> and the `sample_selector` ophyd objects.  In this way, the emitted documents will automatically contain both the image data from the detector and the sample number associated with this data (important metadata).

In [ ]:
RE(select_sample(0))
RE(count([sample_selector, detector]), callback)

If everything worked, you should have seen a pattern automatically appear on the visualization, with the associated sample number ("Sample 0") in the image title.  This data is pulled from the streaming documents emitted by the RunEngine - we didn't have to go get them and fully process them later!

Note that if you look at the immidiate output of the cell, you should see a long-string of letters and numbers.  This serves as a unique identifier (uuid) for that particular measurment that will never be repeated.  These uuids can be used as book-keeping identifiers for later data lookup, but that is beyond the scope of this tutorial.

So, what happens if we measure the same sample again?

In [ ]:
RE(count([sample_selector, detector]), callback)

The <code>stream_to_figures</code> callback we have setup here contains logic to automatically average additional measurments of the same sample together.  As such, you may have noticed that the image quality improved slightly when this second measurment was performed.  The "N_shots" quantity in the plot title also reflects this number of shots on sample increasing.

You can try re-running that cell several times if you'd like, but we can expect little visual change.  This is because sample 0 happens to be a 'strong' scatterer, and it's resultant signal-to-noise quality is good over the background.  In fact, even a single exposure produced an image with sufficient contrast to interpret scientifically.

Next, let's set up a new figure, move to the next sample, and take a measurment.

In [ ]:
fig, axes = plt.subplots(squeeze=False, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes, start_at=1)

In [ ]:
RE(select_sample(1))

In [ ]:
fig, axes = plt.subplots(squeeze=False, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes, start_at=1)

In [ ]:
RE(count([sample_selector, detector]), callback)

This sample happens to be "bad". It gives a weak signal (low signal-to-noise ratio). A single exposure does not produce sufficient contrast, but we can take additional exposures. The visualization above will display the average of all the exposures of this sample.

In [ ]:
RE(count([sample_selector, detector]), callback)

## Acquire images for more than one sample

Here, we make a Figure that display images for both the first (good) and second (bad) sample. As we acquire images, we will see them side by side here.

In [ ]:
fig, axes = plt.subplots(1, 2, squeeze=False, constrained_layout=True, figsize=(5, 3))
callback = stream_to_figures(fig, axes)

In [ ]:
RE(select_sample(0))

In [ ]:
RE(count([sample_selector, detector]), callback)

In [ ]:
RE(select_sample(1))

In [ ]:
RE(count([sample_selector, detector]), callback)

In [ ]:
RE(count([sample_selector, detector]), callback)

## Write a custom Bluesky "plan" to sweep samples

In [ ]:
def sequential_sweep(total_shots):
    "Sweep over the samples in order. Take up to `total_shots` shots."
    for shot in range(total_shots):
        yield from select_sample(shot % 9)
        yield from count([sample_selector, detector])

In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

Our simulated detector has a simulated "delay", standing in for the exposure and readout time in a real detector. Here we'll make it go faster so the following examples runs faster.

In [ ]:
detector.delay = 0.1

In [ ]:
unique_ids = RE(sequential_sweep(total_shots=18), callback)

## Use `bluesky-adaptive` to let an Agent drive the experiment

It is have access to the data as it is acquired and use this to decide when to move to the next sample.

These particular agents are aware of a "budget" of time for this experiment. They aim to make the most efficient use of the available time to obtain high-constrant, interpretable images.

### Agent 1: Naive Agent

This will do effectively same thing we just did above---sequential sweeps---but it will do so using the `bluesky-adaptive` machinery.

In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

In [ ]:
from utils.adaptive_recommendations import NaiveAgent

unique_ids = RE(
    with_agent(NaiveAgent(9), max_shots=70),
    callback,
)

### Agent 2: Reinforcement Learning Agent



In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

In [ ]:
from utils.adaptive_recommendations import RLAgent
detector.delay = 1
unique_ids = RE(
    with_agent(RLAgent(9, 'tf_models/bluesky-tutorial/saved_models'), max_shots=90),
    callback,
)

### Agent 3: "Cheating" (Omniscient) Agent

This agent is told *a priori* which samples are good and which are bad.

In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

In [ ]:
from utils.adaptive_recommendations import CheatingAgent

unique_ids = RE(
    with_agent(CheatingAgent(9), max_shots=50),
    callback,
)